In [1]:
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights import *
from SmoothSparseWeights_Alternating import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=5
# handle_num=11


# exampleName="cow"
# poses_folder="poses-2"
# poses_ind=5
# clip_k=7
# handle_num=15


exampleName="wolf"
poses_folder="poses-1"
poses_ind=2
clip_k=18
handle_num=21


objective_choice=0


weights = {
           'W_sparse': 0.0,
           'W_spatial': 0.0,
           'W_sum': 100000.0
          }

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/wolf-all/poses-1/wolf-1.obj', './models/wolf-all/poses-1/wolf-2.obj', './models/wolf-all/poses-1/wolf-3.obj', './models/wolf-all/poses-1/wolf-4.obj', './models/wolf-all/poses-1/wolf-5.obj', './models/wolf-all/poses-1/wolf-6.obj']
(5075, 24)
(5075, 18)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

#### this is to make sure the 3*4 or 4*3 error does not happen. ???
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])

print (grad_zero_indices[:len(endmembers)*3])

print (mixing_weights_clip.ravel()[grad_zero_indices].sum())

(21, 72)
(5075, 21)
[[0.1589514  0.0000593  0.00474596 0.00048292 0.0000278  0.03343458
  0.07699286 0.17446554 0.03199314 0.00009963 0.01256247 0.12470764
  0.09002658 0.0000002  0.00000122 0.00001448 0.01316137 0.00123948
  0.00000147 0.00000064 0.27703134]
 [0.15856047 0.00005942 0.00472487 0.0004812  0.00002775 0.03355878
  0.07708042 0.17432981 0.03208895 0.00009982 0.01256983 0.12475939
  0.08994991 0.0000002  0.00000122 0.00001445 0.01320413 0.00123941
  0.00000147 0.00000064 0.27724787]
 [0.15566619 0.00006021 0.00463952 0.00047399 0.00002755 0.03405925
  0.07742135 0.17335098 0.03247172 0.00010021 0.01264691 0.12530878
  0.08969101 0.0000002  0.00000124 0.00001417 0.01345853 0.00124074
  0.00000144 0.00000064 0.27936539]]
[[0.1589514  0.0000593  0.00474596 0.00048292 0.0000278  0.03343458
  0.07699286 0.17446554 0.03199314 0.00009963 0.01256247 0.12470764
  0.09002658 0.         0.         0.00001448 0.01316137 0.00123948
  0.00000147 0.         0.27703134]
 [0.15856047 0.0000

In [5]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.
output_prefix="./sparse-smooth-mixing-weights-test/"+exampleName

In [6]:
scale=find_scale(vertices1)
print (scale)

25.309374465859957


In [7]:
### gt test
val=objective_func_vector(mixing_weights.ravel(), vertices1, vertices2, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

1.4503569839497157e-12
2.90071396789943e-09


In [8]:
val=objective_func_vector(mixing_weights_clip.ravel(), vertices1, vertices2, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.7629857489275117
1525.9714978550232


In [9]:
start=time.time()

res, new_endmembers = run(mesh0, mesh1_list, output_prefix, weights, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), grad_zero_indices, initials=mixing_weights_clip.ravel(), choice=objective_choice)

# res, new_endmembers = run(mesh0, mesh1_list, output_prefix, weights, endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), grad_zero_indices, initials=(mixing_weights_clip.ravel() + mixing_weights.ravel())/2.0, choice=objective_choice)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

(5075, 24)
(5075, 18)
####loop:  0
####solve mixing weights
False
738.7778275266768
####solve endmembers
True
####endmember changes:  [ 314.42655213   49.03000395  788.35823754  459.80808113   23.16898584
 1358.09474698  985.90560855  147.00349684 1581.69076286   68.07383151
 1502.14858665  875.10228886  946.04288833    0.00730196    0.68480588
   15.57439939  869.15548237  420.91991515    3.76489291    1.9707552
  305.18604976]
####loop:  1
####solve mixing weights
False
73.97711638900105
####solve endmembers
True
####endmember changes:  [ 81.92509684   3.29946168  72.12123962  33.27596023   2.1514948
 155.63448652 148.48613201  40.1193665  191.37891483   4.33869518
 100.15550154 196.67904948 121.7271522    0.00036443   0.07127127
   1.13257946  97.65706982  35.68553167   0.255598     0.12276258
  65.79947022]
####loop:  2
####solve mixing weights
False
51.47641474412458
####solve endmembers
True
####endmember changes:  [40.44148096  0.72474831 19.75570908  7.73484531  0.52100736 47.4

In [10]:
val=objective_func_vector(res.ravel(), vertices1, vertices2, new_endmembers, mixing_weights.ravel(), mixing_weights_clip.ravel(), choice=objective_choice)
print (np.sqrt(np.square(val).sum()/(len(val)*scale*scale)))
recovered_vertices2=recover_vertices(res, vertices1, new_endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.023743661502822002
47.487323005643994


In [11]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print (res.ravel()[grad_zero_indices].sum()) ### should be zero
print (res.sum())
print ((((1-res.sum(axis=-1))**2).sum()/len(endmembers))*weights["W_sum"]) ##W_sum term

[[0.15893323 0.00001608 0.00471655 0.00063381 0.00005465 0.03341662
  0.07697542 0.17444708 0.03196736 0.00011715 0.01253146 0.12469068
  0.09000966 0.         0.         0.00002985 0.01313819 0.00130423
  0.00000338 0.         0.27701311]
 [0.1585423  0.00001626 0.00469566 0.00063173 0.00005464 0.03354081
  0.07706304 0.17431137 0.03206294 0.00011746 0.01253934 0.12474247
  0.08993302 0.         0.         0.00002981 0.0131809  0.00130374
  0.00000337 0.         0.27722964]
 [0.15564725 0.00001642 0.00460948 0.00063405 0.00005401 0.03404091
  0.07740302 0.17333175 0.03244545 0.00011841 0.01261362 0.12529105
  0.08967342 0.         0.         0.00003009 0.01343407 0.00130554
  0.00000336 0.         0.2793464 ]]
0.0
5075.001962268251
0.0002873412134994578


In [12]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

1.1889981248253427e-11
0.0 0.934863730899431


In [13]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [14]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/wolf-all/wolf.DMAT']
(5075, 21)
5.575027928616619e-29


In [15]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

# gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/wolf-all/poses-1/wolf-1.Tmat', './models/wolf-all/poses-1/wolf-2.Tmat', './models/wolf-all/poses-1/wolf-3.Tmat', './models/wolf-all/poses-1/wolf-4.Tmat', './models/wolf-all/poses-1/wolf-5.Tmat', './models/wolf-all/poses-1/wolf-6.Tmat']
(21, 72)
(6, 21, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.972544   0.227903  -0.0470963  0.885167 ]
 [-0.154564   0.783858   0.601396  -0.609624 ]
 [ 0.173977  -0.577605   0.797562   0.81869  ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.84512     0.00489601 -0.534555   -1.82441   ]
 [ 0.267749    0.861615    0.431196    1.16584   ]
 [ 0.462692   -0.507538    0.726857   -0.924649  ]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [16]:
# #### do a groundtruth test.
# val=objective_func_vector(gt_mixing_weights.ravel(), vertices1, vertices2, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), choice=objective_choice)
# print ((abs(val)).max(), (abs(val)).min())
# print (np.square(val).sum()/(len(val)))

# res=run(mesh0, mesh1_list, output_prefix, weights, gt_handles, gt_mixing_weights.ravel(), gt_mixing_weights.ravel(), grad_zero_indices, initials=gt_mixing_weights.ravel(), choice=objective_choice)
# res=res.reshape(gt_mixing_weights.shape)

In [17]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]
recovered_handle_position=mixing_weights.T.dot(Vertices)/mixing_weights.T.sum(axis=-1)[...,np.newaxis]
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]
postprocessed_handle_position=res.T.dot(Vertices)/res.T.sum(axis=-1)[...,np.newaxis]

%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], c='y', s=1, edgecolors='none')

plt.show()

Using matplotlib backend: MacOSX
